# Homework 5: Boosting and PCA


This assignment is due on Moodle by **11:59pm on Friday November 22**. 
Your solutions to theoretical questions should be done in Markdown/MathJax directly below the associated question.
Your solutions to computational questions should include any specified Python code and results 
as well as written commentary on your conclusions.
Remember that you are encouraged to discuss the problems with your instructors and classmates, 
but **you must write all code and solutions on your own**. For a refresher on the course **Collaboration Policy** click [here](https://github.com/BoulderDS/CSCI5622-Machine-Learning/blob/master/info/syllabus.md#collaboration-policy).

**NOTES**: 

- Do **NOT** load or use any Python packages that are not available in Anaconda (Version: 2019.07) with Python 3.7. 
- Some problems with code may be autograded.  If we provide a function API **do not** change it.  If we do not provide a function API then you're free to structure your code however you like. 
- Submit only this Jupyter notebook to Moodle.  Do not compress it using tar, rar, zip, etc.
- **Unzip the files in data folder**


Name: Andrew Gerlach ange6809

In [ ]:
import math
import pickle
import gzip
import numpy as np
import pandas
import matplotlib.pylab as plt
%matplotlib inline

[40 points] Problem 1 - Principal Component Analysis
---

In this problem you'll be implementing Dimensionality reduction using Principal Component Analysis technique. 

The gist of PCA Algorithm to compute principal components is follows:
- Calculate the covariance matrix X of data points.
- Calculate eigenvectors and corresponding eigenvalues.
- Sort the eigenvectors according to their eigenvalues in decreasing order.
- Choose first k eigenvectors which satisfies target explained variance.
- Transform the original n dimensional data points into k dimensions.

The skeleton for the `PCA` class is below. Scroll down to find more information about your tasks as well as unit tests.

In [ ]:
class PCA:
    def __init__(self, target_explained_variance=None):
        """
        explained_variance: float, the target level of explained variance
        """
        self.target_explained_variance = target_explained_variance
        self.feature_size = -1

    def standardize(self, X):
        """
        standardize features using standard scaler
        :param m X n: features data
        :return: standardized features
        """
        # YOUR CODE HERE
        X_T = X.T
        X_std = np.zeros(shape = X_T.shape)
        for i in range(len(X_T)):
            curr_mean = np.mean(X_T[i])
            curr_std = np.std(X_T[i])
            std_row = (X_T[i] - curr_mean) / curr_std
            X_std[i] = std_row
        return X_std.T

    def compute_mean_vector(self, X_std):
        """
        compute mean vector
        :param X_std: data
        :return n X 1 matrix: mean vector
        """
        # YOUR CODE HERE
        mean_vector = np.zeros(shape = (X_std.shape[1],))
        
        for i in range(X_std.shape[1]):
            curr_sum = sum(X_std[:,i])
            mean_vector[i] = curr_sum / len(X_std[:,i])
            
        return mean_vector
             

    def compute_cov(self, X_std, mean_vec):
        """
        Covariance using mean, (don't use any numpy.cov)
        :param X_std:
        :param mean_vec:
        :return n X n matrix:: covariance matrix
        """
        # YOUR CODE HERE
        cov_matrix = np.zeros(shape = (len(mean_vec), len(mean_vec)))
        temp = np.zeros(shape = X_std.shape)
        temp = X_std - mean_vec
        for i in range(temp.shape[1]):
            cov_i = temp[:,i]
            for j in range(temp.shape[1]):
                cov_j = temp[:,j]
                cov_i_j = np.dot(cov_i, cov_j) / (temp.shape[0] - 1)
                cov_matrix[i][j] = cov_i_j
                
        return cov_matrix
            

    def compute_eigen_vector(self, cov_mat):
        """
        Eigenvector and eigen values using numpy
        :param cov_mat:
        :return: (eigen_vector,eigen_values)
        """
        # YOUR CODE HERE
        return np.linalg.eig(cov_mat)

    def compute_explained_variance(self, eigen_vals):
        """
        sort eigen values and compute explained variance.
        explained variance informs the amount of information (variance)
        can be attributed to each of  the principal components.
        :param eigen_vals:
        :return: explained variance.
        """
        # YOUR CODE HERE
        sum_of_eigen_vals = np.sum(eigen_vals)
        
        #Fill the list with the variances in decreasing order
        explained_variance = [ev/sum_of_eigen_vals for ev in sorted(eigen_vals, reverse = True)]
        
        return explained_variance
        

    def cumulative_sum(self, var_exp):
        """
        return cumulative sum of explained variance.
        :param var_exp: explained variance
        :return: cumulative explained variance
        """
        return np.cumsum(var_exp)

    def compute_weight_matrix(self, eig_pairs, cum_var_exp):
        """
        compute weight matrix of top principal components conditioned on target
        explained variance.
        (Hint : use cumulative explained variance and target_explained_variance to find
        top components)
        
        :param eig_pairs: list of tuples containing eigenvector and eigen
        values
        :param cum_var_exp: cumulative explained variance by features
        :return: weight matrix
        """
        # YOUR CODE HERE
        weight_matrix = []
        #sort the eigen_pairs list in descending order
        eig_pairs_sorted = sorted(eig_pairs, key = lambda x : x[1], reverse = True)
        for i in range(len(cum_var_exp)):
            if cum_var_exp[i] <= self.target_explained_variance:
                weight_matrix.append(eig_pairs_sorted[i][0]) 
        self.feature_size = len(weight_matrix)
        return np.transpose(weight_matrix)
    
    def transform_data(self, X_std, matrix_w):
        """
        transform data to subspace using weight matrix
        :param X_std: standardized data
        :param matrix_w: weight matrix
        :return: data in the subspace
        """
        return X_std.dot(matrix_w)

    def fit(self, X):
        """
        entry point to the transform data to k dimensions
        standardize and compute weight matrix to transform data.
        :param   m X n dimension: train samples
        :return  m X k dimension: subspace data.
        """
        self.feature_size = X.shape[1]
        
        # YOUR CODE HERE
        X_std = self.standardize(X)
        
        mean_vector = self.compute_mean_vector(X_std)
        cov_matrix = self.compute_cov(X_std, mean_vector)
        eigen_vals,eigen_vecs = self.compute_eigen_vector(cov_matrix)
        
        eigen_vecs = eigen_vecs.T
        
        exp_variance = self.compute_explained_variance(eigen_vals)

        cum_sum = self.cumulative_sum(exp_variance)

        eig_pairs = []
        for i in range(len(eigen_vals)):
            eig_pairs.append((eigen_vecs[i], eigen_vals[i]))
            
        matrix_w = self.compute_weight_matrix(eig_pairs, cum_sum)

        return self.transform_data(X_std = X_std, matrix_w = matrix_w)


**Part 1 [20 points]:**  Your task involves implementing helper functions to compute mean, 
covariance, eigenvector and weights.

Complete `fit` to use all helper functions to find reduced dimension data.

In [ ]:
from tests import tests
tests.run_test_suite("prob 1", PCA)

**Part 2 [5 points]:**   Run PCA on fashion mnist dataset to reduce the dimension of the data.

fashion mnist data consists of samples with 784 dimensions.

Report the reduced dimension $k$ for target explained variance of **0.99**

In [ ]:
X_train = pickle.load(open('./data/mnist/train_images.pkl','rb'))
y_train = pickle.load(open('./data/mnist/train_image_labels.pkl','rb'))
X_train = X_train[:15000]
y_train = y_train[:15000]

In [ ]:
# YOUR CODE HERE
pca = PCA(target_explained_variance = 0.99)
X_train_updated = pca.fit(X_train)

In [ ]:
print("Data features went from {0} to {1} ".format(X_train.shape[1] , pca.feature_size))

**Part 3 [5 points]:** Run scikit-learn SVM Classifier (refer previous homework) on the reduced dimension data with approrpiate kernel and C.

Report the accuracy on test dataset.

In [ ]:
from sklearn.model_selection import train_test_split
X_t, X_test, y_t, y_test = train_test_split(X_train_updated, y_train, test_size=0.2, random_state=5622)

In [ ]:
# YOUR CODE HERE
from sklearn.svm import SVC
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.pipeline import make_pipeline, Pipeline
from sklearn.model_selection import GridSearchCV

svm = SVC(degree = 3, gamma = 0.1, kernel = 'poly')
svm.fit(X_t, y_t)
print("Score on test data: {}".format(svm.score(X_test, y_test)))
# print(svm.predict(X_test))
# print(svm.score(X_test, y_test))
# print(confusion_matrix(y_test, y_pred))
# print(classification_report(y_test, y_pred))

# svc = make_pipeline(SVC())

# parameters = [{'svc__kernel': ['linear'], 'svc__C': [0.01, 1.0, 10.0]}, 
#  {'svc__kernel' : ['poly'], 'svc__degree' : [2, 3], 'svc__gamma' : [0.1, 0.5, 1.0] },
#  {'svc__kernel': ['rbf'], 'svc__gamma': [0.1, 0.5, 1.0]}]

# grid_svc = GridSearchCV(svc, param_grid = parameters, scoring = 'accuracy')

# find_best = grid_svc.fit(X_t, y_t)

**Part 4 [10 points]:** Repeat the same experiment for different values of target explained variance between: **[0.8-1.0]** with increments of $0.04$, provide the reduced dimension size for each, and then:

- Plot the graph of accuracy vs target explained variance.
- Plot the graph of the number of components vs target explained variance.

In [ ]:
X_train = pickle.load(open('./data/mnist/train_images.pkl','rb'))
y_train = pickle.load(open('./data/mnist/train_image_labels.pkl','rb'))
X_train = X_train[:15000]
y_train = y_train[:15000]

target_explained_variances = []
numbers_of_components = []
accuracies = []

for target_variance in np.arange(0.8, 1.01, 0.04):
    # YOUR CODE HERE
    target_explained_variances.append(target_variance)
    curr_pca = PCA(target_explained_variance = target_variance)
    X_train_updated = curr_pca.fit(X_train)
    numbers_of_components.append(X_train_updated.shape[1])
    X_t, X_test, y_t, y_test = train_test_split(X_train_updated, y_train, test_size=0.2, random_state=5622)
    svc = SVC(degree = 3, gamma = 0.1, kernel = 'poly')
    svc.fit(X_t, y_t)
    accuracies.append(svc.score(X_test,y_test))

print("TEVs: ", target_explained_variances)
print("# of Components: ", numbers_of_components)
print("Accuracies: ", accuracies)

In [ ]:
# Make plots here
# YOUR CODE HERE
fig, ax = plt.subplots(nrows=1, ncols=1, figsize=(12,6))
ax.plot(target_explained_variances,  accuracies, color="black")
ax.set_xlabel("Target Explained Variances", fontsize=16)
ax.set_ylabel("Testing Accuracy", fontsize=16)
plt.show()

In [ ]:
# Make plots here
# YOUR CODE HERE
fig, ax = plt.subplots(nrows=1, ncols=1, figsize=(12,6))
ax.plot(target_explained_variances,  numbers_of_components, color="red", label = "Number of Components vs TEV")
ax.set_xlabel("Target Explained Variances", fontsize=16)
ax.set_ylabel("Number of Components", fontsize=16)
plt.show()

Discuss your observations below.

As the Target Explained Variance increases the accuracy and the number of features increases.

[20 points] Problem 2 - Statistical PCA for non-zero mean random variables. 
---
Let $x \in R^D$ be a random vector. Let $\mu_x$ = $E(x)$  $ \in R^D$ and $\sum_x = E(x − \mu)(x − \mu)$ be mean an covariance of x respectivley. 

We define *Principal components* of $x$ as $v_i$. Assuming that the eigenvalues of $\sum_x$ are different from each other

show that

1) $v_1$ is the eigenvector of $\sum_x$ corresponding to its largest eigenvalue.

2) $v_2^T v_1$ = 0 and $v_2$ is the eigenvector of $\sum_x$ corresponding to its second largest eigenvalue.

1)
The variance of each feature is given by $w^{T}$$C^{X}$$w$. Because we want he largest variance for the first principal component this can be found via:  $max_{w}$ $w^{T}$$C^{X}$$w$ s.t. $w^{T}$$w$ = 1. Using the Lagrangian $L(w,\lambda) = w^{T}C^{X}w - \lambda(w^{T}w - 1)$

Set the gradient of L equal to zero to maximize w: $\nabla_{w}L = 2C^{X}w - 2\lambda w = 0$ 

From this we can see the way to maximize $w$ is to pick the largest value of lambda which will correspond to the first principal component, $v_1$.



2)
The same can be done to find the second principal component, $v_2$. The covariance matrix can be altered to remove $v_1$: $C^{X} = \sum_{i = 2}^{D}\lambda v_i v_i^{T}$ 

because $C^{X} = \sum_{i = 2}^{D}\lambda v_i v_i^{T} = \frac{1}{m-1}(X^{T}X - (m-1)\lambda v_i v_i^{T})$ ($v_1$ is removed).

Then, the saem process as part 1 will find $v_2$ which will correspond to the second highest eigen value and be the second principal component.

$v_{2}^{T} v_1 = 0$ because the covariance matrix is a symmetric matrix and in symmetric matrices the eigen vectors are orthogonal. The dot product between any 2 orthogonal vecotrs is 0.

[40 points] Problem 3  - Decision Tree Ensembles: Bagging and Boosting
---

We are going to predict house price using decision tree ensembles.

In this Regression problem, we compare Decision trees and it's ensembles - bagging and Boosting on House Price prediction dataset : https://www.kaggle.com/c/house-prices-advanced-regression-techniques/data

Make use of standard Regression API's of  Decision tree ensembles from sklearn to predict the house price. http://scikit-learn.org/stable/modules/classes.html#module-sklearn.ensemble

**Part 1 [20 points]:**
Complete the `ensemble_test` class to fit appropriate model recieved as parameter and store the test accuracy.
Later we will also use `ensemble_test` class to plot score, metric and time taken to fit the data.

In [ ]:
from time import time
from sklearn.metrics import explained_variance_score, precision_score
import pandas as pd
import matplotlib.pyplot as pyp

class EnsembleTest:
    """
        Test multiple model performance
    """

    def __init__(self, X_train, y_train, X_test, y_test, type='regression'):
        """
        initialize data and type of problem
        :param X_train:
        :param y_train:
        :param X_test:
        :param y_test:
        :param type: regression or classification
        """
        self.scores = {}
        self.execution_time = {}
        self.metric = {}
        self.X_train = X_train
        self.y_train = y_train
        self.X_test = X_test
        self.y_test = y_test
        self.type = type
        self.score_name = 'R^2 score' if self.type == 'regression' else 'Mean accuracy score'
        self.metric_name = 'Explained variance' if self.type == 'regression' else 'Precision'

    def fit_model(self, model, name):
        """
        Fit the model on train data.
        predict on test and store score and execution time for each fit.
        :param model: model
        :param name: name of model
        """
        # YOUR CODE HERE
        start = time()
        model_fit = model.fit(self.X_train, y_train)
        self.scores[name] = model_fit.score(X_test, y_test)
        predict = model_fit.predict(X_test)
        finish = time()
        self.execution_time[name] = finish - start
        if self.type == 'regression':
            self.metric[name] = explained_variance_score(predict, self.y_test)
        elif self.type == 'classification':
            self.metric[name] = precision_score(predict, y_test)

    def print_result(self):
        """
            print results for all models trained and tested.
        """
        models_cross = pd.DataFrame({
            'Model'         : list(self.metric.keys()),
             self.score_name     : list(self.scores.values()),
             self.metric_name    : list(self.metric.values()),
            'Execution time': list(self.execution_time.values())})
        print(models_cross.sort_values(by=self.score_name, ascending=False))

    def plot_metric(self):
        """
         plot each metric : time, metric score,scores
        """
        # YOUR CODE HERE
        labels = list(self.metric.keys())
        metrics = list(self.metric.values())
        times = list(self.execution_time.values())
        scores = list(self.scores.values())
        
        x = np.arange(len(labels))
        width = 0.25
        fig, ax = pyp.subplots(nrows=1, ncols=1, figsize=(12,6))
        bar_1 = ax.bar(x - width, times, width, label='Execution Time (s)')
        bar_2 = ax.bar(x, metrics, width, label=self.metric_name)
        bar_3 = ax.bar(x + width, scores, width, label='Score')
        ax.set_xticks(x)
        ax.set_xticklabels(labels)
        ax.legend()
        
        for rect in bar_1:
            height = rect.get_height()
            ax.annotate('{}'.format(round(height,3)),
                    xy=(rect.get_x() + rect.get_width() / 2, height),
                    xytext=(0, 3),  # 3 points vertical offset
                    textcoords="offset points",
                    ha='center', va='bottom')
 
        for rect in bar_2:
            height = rect.get_height()
            ax.annotate('{}'.format(round(height,3)),
                    xy=(rect.get_x() + rect.get_width() / 2, height),
                    xytext=(0, 3),  # 3 points vertical offset
                    textcoords="offset points",
                    ha='center', va='bottom')
            
        for rect in bar_3:
            height = rect.get_height()
            ax.annotate('{}'.format(round(height,3)),
                    xy=(rect.get_x() + rect.get_width() / 2, height),
                    xytext=(0, 3),  # 3 points vertical offset
                    textcoords="offset points",
                    ha='center', va='bottom')
        pyp.tight_layout()
        pyp.show()

In [ ]:
X_train, X_test, y_train, y_test = pickle.load(open('./data/house_predictions/test_train.pkl','rb'))

# create a handler for ensemble_test, use the created handler for fitting different models.
ensemble_handler = EnsembleTest(X_train, y_train, X_test, y_test, type='regression')
from sklearn.tree import DecisionTreeRegressor
decision = DecisionTreeRegressor()
ensemble_handler.fit_model(decision,'decision tree')

**Part 1 A [10 points]:**
Complete the cells below to fit the dataset using `RandomForestRegressor` and `AdaBoostRegressor` with appropriate parameter. Use `n_estimators=1000` for both regressors.

In [ ]:
from sklearn.ensemble import RandomForestRegressor

# YOUR CODE HERE
rfr = RandomForestRegressor(n_estimators = 1000)
ensemble_handler.fit_model(rfr, 'random forest')

In [ ]:
from sklearn.ensemble import AdaBoostRegressor

# YOUR CODE HERE
ada = AdaBoostRegressor()
ensemble_handler.fit_model(ada, 'ada boost')

**Part 1 B [5 points]:** Report results and make plots.

In [ ]:
# Report results here
# YOUR CODE HERE
print(ensemble_handler.print_result())

In [ ]:
# Make plots here
# YOUR CODE HERE
ensemble_handler.plot_metric()

**Part 2 [15 points]:** This is an extension of HW4 problem on sentiment classification over reviews.

Here we make use DecisionTree ensembles to **classify** review as positive or negative

In [ ]:
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import TweetTokenizer
from sklearn.feature_extraction.text import CountVectorizer


In [ ]:
reviews  = pd.read_csv('./data/reviews.csv')
train, test = train_test_split(reviews, test_size=0.2, random_state=4622)
X_train = train['reviews'].values
X_test = test['reviews'].values
y_train = train['sentiment']
y_test = test['sentiment']

**Part 2 A [10 points]:** Perform the following: 

* Create pipeline for `RandomForestClassifier` and `AdaBoostClassifier` as shown for `DecisionTreeClassifier` below. Refer to: http://scikit-learn.org/stable/modules/classes.html#module-sklearn.ensemble
* Fit the reviews dataset on the above models and report the results. (tune parameters of classifier for optimal performance)
* Use `n_estimators = 500` for both classifiers.

In [ ]:
# Define tokenizer and create pipeline.
def tokenize(text): 
    tknzr = TweetTokenizer()
    return tknzr.tokenize(text)
en_stopwords = set(stopwords.words("english")) 
vectorizer = CountVectorizer(
    analyzer = 'word',
    tokenizer = tokenize,
    lowercase = True,
    ngram_range=(1, 2),
    stop_words = en_stopwords,
    min_df=10)

In [ ]:
# create a handler for ensemble_test , use the created handler for fitting different models.
ensemble_classifier_handler = EnsembleTest(X_train,y_train,X_test,y_test,type='classification')

In [ ]:
from sklearn.tree import DecisionTreeClassifier
pipeline_decision_tree = make_pipeline(vectorizer, DecisionTreeClassifier())
ensemble_classifier_handler.fit_model(pipeline_decision_tree,'decision tree classifier')

In [ ]:
ensemble_classifier_handler.print_result()

In [ ]:
from sklearn.ensemble import RandomForestClassifier
# YOUR CODE HERE

rfc = make_pipeline(vectorizer, RandomForestClassifier(n_estimators = 500, 
                                                       criterion = 'gini', 
                                                       max_depth = 30, 
                                                       min_samples_leaf = 1, 
                                                       min_samples_split = 5, 
                                                       oob_score = False))
ensemble_classifier_handler.fit_model(rfc,'random forest classifier')
ensemble_classifier_handler.print_result()


#Find best parameters

#print(rfc.get_params())
# parameters = [
# {'randomforestclassifier__max_depth' : [5, 8, 15, 25, 30],
# 'randomforestclassifier__min_samples_split' : [2, 5, 10, 15, 30, 75, 100],
# 'randomforestclassifier__min_samples_leaf' : [1, 2, 5, 10],
# 'randomforestclassifier__oob_score' : [True, False], 
# 'randomforestclassifier__criterion': ['gini', 'entropy']}]

# gridF = GridSearchCV(rfc, param_grid = parameters, verbose = 1)
# bestF = gridF.fit(X_train, y_train)
# print(bestF.best_params_)

In [ ]:
from sklearn.ensemble import AdaBoostClassifier
# YOUR CODE HERE
abc = make_pipeline(vectorizer, AdaBoostClassifier(n_estimators = 500, learning_rate = 0.25))
ensemble_classifier_handler.fit_model(abc,'adaboost classifier')


#Find best parameters

# print(abc.get_params())
#Trying different learning rates
# parameters_ADA = [
# {'adaboostclassifier__learning_rate': [0.01, 0.1, 0.25, 0.5, 0.75, 1.0]}]

# gridADA = GridSearchCV(abc, param_grid = parameters_ADA, verbose = 1)
# bestADA = gridADA.fit(X_train, y_train)
# print(bestADA.best_params_)

**Part 2 B [5 points]:** Report results and make plots.

In [ ]:
# Report results here
# YOUR CODE HERE
ensemble_classifier_handler.print_result()

In [ ]:
# Make plots here
# YOUR CODE HERE
ensemble_classifier_handler.plot_metric()

**Part 3 [10 points]:** In the following space below discuss at least one advantage and disadvantage for *Random Forest* and *AdaBoost*.

**AdaBoost**

Advantage: Very few parameters to tune, just the number of repetitions.

Disadvantage: Sensitive to noisy data because of the use of weak classifiers.


**Random Forest** 

Advantage: Combining the output of many small decision trees (bagging) results in less overfitting, which improves accuracy overall.

Disadvantage: Generally slow due to the creation of many decision trees (or stumps).

### Optional survey.
***

We are always interested in your feedback. At the end of each homework, there is a simple anonymous feedback [survey](https://forms.gle/T9VKL7UPWjWoYxVu6) to solicit your feedback for how to improve the course.